In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/Users/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/Users/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'PFCA_thermo',
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['1,3_sigmatropic_rearrangement']


In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [5]:
database.kinetics.families

{'1,3_sigmatropic_rearrangement': <ReactionFamily "1,3_sigmatropic_rearrangement">}

In [6]:
# 'Lactone_Formation' 
family_to_train = "1,3_sigmatropic_rearrangement"
family = database.kinetics.families[family_to_train]

In [7]:
family.clean_tree()

In [8]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

┌ Warning: Replacing docs for `ReactionMechanismSimulator.getpairs :: Tuple{T} where T<:ReactionMechanismSimulator.AbstractReaction` in module `ReactionMechanismSimulator`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `ReactionMechanismSimulator.getsimilarity :: Union{Tuple{T2}, Tuple{T}, Tuple{T, T2}} where {T<:ReactionMechanismSimulator.AbstractSpecies, T2<:ReactionMechanismSimulator.AbstractSpecies}` in module `ReactionMechanismSimulator`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: DiffEqOperators.jl is in the process of being deprecated.
│ 
│        - For automated PDE discretization, see MethodOfLines.jl
│ 
│        - For MatrixFreeOperators, and other non-derivative operators, see SciMLOperators.jl
│ 
│        - For VecJacOperators and JacVecOperators, see SparseDiffTools.jl
└ @ DiffEqOperators ~/miniconda3/envs/rmg_env/share/julia/packages/DiffEqOperators/lHq9u/src/DiffEqOperators.jl:61
┌ Warning: Replacing docs for `ReactionMechanismSimulator.plotrops :: Union{Tupl

358.71815299987793


In [9]:
len(family.groups.entries)

73

In [10]:
family.groups.entries

{'Root': <Entry index=0 label="Root">,
 'Root_1R!H->C': <Entry index=1 label="Root_1R!H->C">,
 'Root_N-1R!H->C': <Entry index=2 label="Root_N-1R!H->C">,
 'Root_1R!H->C_1C-inRing': <Entry index=3 label="Root_1R!H->C_1C-inRing">,
 'Root_1R!H->C_N-1C-inRing': <Entry index=4 label="Root_1R!H->C_N-1C-inRing">,
 'Root_N-1R!H->C_Ext-3R!H-R': <Entry index=5 label="Root_N-1R!H->C_Ext-3R!H-R">,
 'Root_N-1R!H->C_Ext-4R!H-R': <Entry index=6 label="Root_N-1R!H->C_Ext-4R!H-R">,
 'Root_1R!H->C_1C-inRing_2R!H->N': <Entry index=7 label="Root_1R!H->C_1C-inRing_2R!H->N">,
 'Root_1R!H->C_1C-inRing_N-2R!H->N': <Entry index=8 label="Root_1R!H->C_1C-inRing_N-2R!H->N">,
 'Root_1R!H->C_N-1C-inRing_2R!H->N': <Entry index=9 label="Root_1R!H->C_N-1C-inRing_2R!H->N">,
 'Root_1R!H->C_N-1C-inRing_N-2R!H->N': <Entry index=10 label="Root_1R!H->C_N-1C-inRing_N-2R!H->N">,
 'Root_N-1R!H->C_Ext-3R!H-R_Ext-3R!H-R': <Entry index=11 label="Root_N-1R!H->C_Ext-3R!H-R_Ext-3R!H-R">,
 'Root_N-1R!H->C_Ext-3R!H-R_Ext-2R!H-R': <Entr

In [11]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.0074770450592041016


In [12]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([182.551,212.529,232.841,248.154,269.955,282.626,294.993],'J/(mol*K)'), H298=(-1467.92,'kJ/mol'), S298=(554.213,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(291.007,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-C2sCs) + group(Cs-CsF1sF1sO2s) + group(Cs-CsF1sF1sF1s) + longDistanceInteraction_noncyclic(Cs(F)3-Cs(F)2) + group(Cs-C2sF1sF1sF1s) + group(CsJ2_singlet-CsH)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([182.551,212.529,232.841,248.154,269.955,282.626,294.993],'J/(mol*K)'), H298=(-1467.92,'kJ/mol'), S298=(554.213,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(291.007,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-C2sCs) + group(Cs-CsF1sF1sO2s) + group(Cs-CsF1sF1sF1s) + longDistanceInteraction_noncyclic(Cs(F)3-Cs(F)2) + group(Cs-C2s

1.3910479545593262


In [13]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([182.551,212.529,232.841,248.154,269.955,282.626,294.993],'J/(mol*K)'), H298=(-1467.92,'kJ/mol'), S298=(554.213,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(291.007,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-C2sCs) + group(Cs-CsF1sF1sO2s) + group(Cs-CsF1sF1sF1s) + longDistanceInteraction_noncyclic(Cs(F)3-Cs(F)2) + group(Cs-C2sF1sF1sF1s) + group(CsJ2_singlet-CsH)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([182.551,212.529,232.841,248.154,269.955,282.626,294.993],'J/(mol*K)'), H298=(-1467.92,'kJ/mol'), S298=(554.213,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(291.007,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-C2sCs) + group(Cs-CsF1sF1sO2s) + group(Cs-CsF1sF1sF1s) + longDistanceInteraction_noncyclic(Cs(F)3-Cs(F)2) + group(Cs-C2s

1.326225757598877


In [14]:
len(templateRxnMap)

73

In [15]:
family.clean_tree_rules()

In [16]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/Users/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3636: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])


ending first loop
position 1, 0.0
term 1: 406598.0, term 2: -59693.067332000035
position 2: 346904.932668
position 3: 346904.932668
position 4: 346904.932668
position 6: 271878.2610451951
position 9: 271878.2610451951
position 10: 271878.2610451951
ending first loop
ArrheniusBM(A=(6.1544e+10,'s^-1'), n=1.34775, w0=(693.81,'kJ/mol'), E0=(271.878,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")
ending first loop
position 1, 0.0
term 1: 192369.0, term 2: -18560.443750000006
position 2: 173808.55625
position 3: 173808.55625
position 4: 173808.55625
position 6: 152636.96360284914
position 9: 152636.96360284914
position 10: 152636.96360284914
ending first loop
ArrheniusBM(A=(3.57015e+11,'s^-1'), n=0.576299, w0=(707,'kJ/mol'), E0=(152.637,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100

/Users/khalil.nor/miniconda3/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:488: RuntimeWarning: divide by zero encountered in log
  return transform * (func(xdata, *params) - ydata)
/Users/khalil.nor/miniconda3/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


position 6: 7870.0
position 9: 7870.0
position 10: 7870.0
ending first loop
ArrheniusBM(A=(3.59068e+184,'s^-1'), n=-42.9202, w0=(787,'kJ/mol'), E0=(7.87,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")
ending first loop
position 1, 0.0
term 1: 121057.36377031561, term 2: 18560.443750000006
position 2: 139617.80752031563
position 3: 139617.80752031563
position 4: 139617.80752031563
position 6: 155612.97081165385
position 9: 155612.97081165385
position 10: 155612.97081165385
ending first loop
ArrheniusBM(A=(9.74444e+12,'s^-1'), n=-0.195371, w0=(707,'kJ/mol'), E0=(155.613,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")
ending first loop
position 1, 0.0
term 1: 321430.0, term 2: -54882.22881999999
position 2: 266547.77118000004
term 1: 359894.0, term 2: -4755

/Users/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:4643: RuntimeWarning: overflow encountered in exp
  kin = ArrheniusBM().fit_to_reactions(rxns, recipe=recipe)


position 9: 7870.0
position 10: 7870.0
ending first loop
ArrheniusBM(A=(inf,'s^-1'), n=-98.0322, w0=(787,'kJ/mol'), E0=(7.87,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")
ending first loop
position 1, 0.0
term 1: 446512.0, term 2: -51560.155832000004
position 2: 394951.844168
position 3: 394951.844168
position 4: 394951.844168
position 6: 334126.60123781237
position 9: 334126.60123781237
position 10: 334126.60123781237
ending first loop
ArrheniusBM(A=(1.04679e+11,'s^-1'), n=1.26667, w0=(789.904,'kJ/mol'), E0=(334.127,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]""")
ending first loop
position 1, 0.0
term 1: 103611.0, term 2: 14438.314650000015
position 2: 118049.31465000001
term 1: 146289.74308544458, term 2: 15927.41585000002
position 2: 280266.47358544

/Users/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:4655: RuntimeWarning: overflow encountered in exp
  ) for i, rxn in enumerate(rxns)


position 9: 7870.0
position 10: 7870.0
position 1, 0.0
term 1: -1251341.4496971779, term 2: 374609.37044
position 2: -876732.0792571779
position 3: -876732.0792571779
position 4: -876732.0792571779
position 5: 7870.0
position 6: 7870.0
position 9: 7870.0
position 10: 7870.0
[inf inf]
ending second loop


/Users/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:4665: RuntimeWarning: invalid value encountered in subtract
  s = np.sqrt(np.dot(ws, (dlnks - mu) ** 2) / (V1 - V2 / V1))


ending first loop
position 1, 0.0
term 1: 351456.0, term 2: 12276.490999999995
position 2: 363732.491
term 1: 103611.0, term 2: 14438.314650000015
position 2: 481781.80565
term 1: 191066.0, term 2: 7998.540707752851
position 2: 680846.3463577528
term 1: 262426.0, term 2: 25572.616592747276
position 2: 968844.9629505001
term 1: 209675.0, term 2: 10672.920676520996
position 2: 1189192.883627021
term 1: 485946.49600000004, term 2: -30006.692832873436
position 2: 1645132.6867941476
term 1: 325555.36640000006, term 2: 79074.6905
position 2: 2049762.7436941476
term 1: 386590.3032, term 2: -12276.490999999995
position 2: 2424076.5558941476
term 1: 203328.59279999998, term 2: 4789.919999999998
position 2: 2632195.0686941477
term 1: 367166.08320000005, term 2: -14398.329999999994
position 2: 2984962.821894148
term 1: 397097.4909173832, term 2: -12276.490999999995
position 2: 3369783.821811531
term 1: 146289.74308544458, term 2: 15927.41585000002
position 2: 3532000.9807469756
term 1: 121057.363

In [17]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.007175922393798828


In [18]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([182.551,212.529,232.841,248.154,269.955,282.626,294.993],'J/(mol*K)'), H298=(-1467.92,'kJ/mol'), S298=(554.213,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(291.007,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-C2sCs) + group(Cs-CsF1sF1sO2s) + group(Cs-CsF1sF1sF1s) + longDistanceInteraction_noncyclic(Cs(F)3-Cs(F)2) + group(Cs-C2sF1sF1sF1s) + group(CsJ2_singlet-CsH)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([182.551,212.529,232.841,248.154,269.955,282.626,294.993],'J/(mol*K)'), H298=(-1467.92,'kJ/mol'), S298=(554.213,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(291.007,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-C2sCs) + group(Cs-CsF1sF1sO2s) + group(Cs-CsF1sF1sF1s) + longDistanceInteraction_noncyclic(Cs(F)3-Cs(F)2) + group(Cs-C2s

position 1, 0.0
term 1: 169256.13439783317, term 2: 59693.067332000006
position 2: 228949.20172983318
term 1: 171455.1849071861, term 2: 47550.762520000004
position 2: 447955.14915701933
term 1: 234571.46715769861, term 2: 51560.155832000004
position 2: 734086.7721467179
position 3: 244695.59071557262
position 4: 244695.59071557262
position 6: 280179.25573377236
position 9: 280179.25573377236
position 10: 280179.25573377236
position 1, 0.0
term 1: 103611.0, term 2: 14438.314650000008
position 2: 118049.31465000001
term 1: 121057.36377031561, term 2: 18560.443750000006
position 2: 257667.12217031565
term 1: 146289.74308544458, term 2: 15927.41585000002
position 2: 419884.28110576025
position 3: 139961.4270352534
position 4: 139961.4270352534
position 6: 152812.54765975577
position 9: 152812.54765975577
position 10: 152812.54765975577
position 1, 0.0
term 1: 446512.0, term 2: -51560.155832000004
position 2: 394951.844168
term 1: 425892.86452322314, term 2: -11819.247712000002
position 2:

/Users/khalil.nor/miniconda3/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:488: RuntimeWarning: divide by zero encountered in log
  return transform * (func(xdata, *params) - ydata)
/Users/khalil.nor/miniconda3/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [19]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

/Users/khalil.nor/Code/RMG-database/input/kinetics/families/1,3_sigmatropic_rearrangement


In [20]:
family.save(save_path)

In [21]:
#also save to this folder 
save_here = '.'
family.save(save_here)

In [ ]:
# notebook ran to completion for third retraining on Oct 8th 2024, 3:00 pm

In [ ]:
#let's check to see what the fit is like at the end nodes

In [ ]:
from rmgpy.kinetics.uncertainties import RateUncertainty
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM, Arrhenius
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib
import rmgpy.constants as constants

In [ ]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'PFCA_thermo',
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ["Lactone_Formation"]

database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )

In [ ]:
database.kinetics.families

In [ ]:
# "1,2_Insertion_carbene"
family_to_train = "Lactone_Formation"
family = database.kinetics.families[family_to_train]

In [ ]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

In [ ]:
Let's look at some of the parent nodes (with 2 reactions) and their child nodes (with 1 reaction) to see how the fits compare

In [ ]:
n_2_mapping = [key for key, value in templateRxnMap.items() if len(value)==2] #mapping of all nodes with 3 or less training rxns (parents)
n_1_mapping = [key for key, value in templateRxnMap.items() if len(value)==1] #mapping of all nodes with 1 training rxn (children)
n_4_mapping = [key for key, value in templateRxnMap.items() if len(value)==4] 
parent_to_child = {}
parent_to_child_4 = {}

for label in n_2_mapping: 
    
    #make the parent node
    parent_node, _ = family.retrieve_original_entry(label)
    
    #now find the children with just 1 reaction
    children_nodes = []
    for test_label in n_1_mapping:
        if label in test_label:
            child_node, _ = family.retrieve_original_entry(test_label)
            children_nodes.append(child_node)
            
            
    if len(templateRxnMap[label])==len(children_nodes): #let's pick out the ones that are both two
        parent_to_child[parent_node.label] = [x.label for x in children_nodes]
        
for label in n_4_mapping: 
    
    #make the parent node
    parent_node, _ = family.retrieve_original_entry(label)
    
    #now find the children with just 1 reaction
    children_nodes = []
    for test_label in n_1_mapping:
        if label in test_label:
            child_node, _ = family.retrieve_original_entry(test_label)
            children_nodes.append(child_node)
            
            
    if len(templateRxnMap[label])==len(children_nodes): #let's pick out the ones that are both two
        parent_to_child_4[parent_node.label] = [x.label for x in children_nodes]
#parent_to_child is a dictionary where key = label of parent and value = label of children

In [ ]:
%matplotlib inline
temperatures = [ 300., 500.,  600.,  700. , 800.,  900., 1000., 1100., 1200., 1500., 2000.]
inverse_temps = [1000/T for T in temperatures]



for parent_label, children_labels in parent_to_child.items():
    
    training_rxns = templateRxnMap[parent_label] 
    for tr_rxn in training_rxns:
        print(tr_rxn)
    BM_entries = [family.retrieve_original_entry(label)[0].data for label in children_labels]

    matches = []
    matched_BM = []

    for BM_kinetics in BM_entries:
        for tr_rxn in training_rxns:
            BM_str_of_float = re.search('([1-9]\.?[0-9][0-9])', str(BM_kinetics.A.value_si)).group(1)
            tr_str_of_float = re.search('([1-9]\.?[0-9][0-9])', str(tr_rxn.kinetics.A.value_si)).group(1)
            if BM_str_of_float==tr_str_of_float:
                #print('yeah')
                matches.append((Reaction(reactants=tr_rxn.reactants, products=tr_rxn.products, kinetics = BM_kinetics), tr_rxn))
                matched_BM.append(BM_kinetics)
    if len(matches)==0:
        print("couldn't find any matches")
        continue
        #skipping because the A's are probably very different
        
    #if you miss matching all of them
    if len(matches)==1:
        leftover_tr_rxn = [tr_rxn for tr_rxn in training_rxns if tr_rxn not in matches[0]][0]
        leftover_BM_kinetics = [BM_kinetics for BM_kinetics in BM_entries if BM_kinetics not in matched_BM][0]
        leftover_BM_reaction = Reaction(reactants=leftover_tr_rxn.reactants, products=leftover_tr_rxn.products, kinetics = leftover_BM_kinetics)
        #print(leftover_BM_reaction.kinetics.A, leftover_tr_rxn.kinetics.A)
        matches.append((leftover_BM_reaction, leftover_tr_rxn))

#     for (BM, tr) in matches: 
#         print(str(BM.kinetics.A.value_si), str(tr.kinetics.A.value_si))
        
    child_node_1_BM_rxn, child_training_rxn_1= matches[0] 
    child_node_2_BM_rxn, child_training_rxn_2 = matches[1] 
    
    #empty lists for collecting data
    parent_node_rate_coefficients_1 = []
    parent_node_rate_coefficients_2 = []

    child_node_1_rate_coefficients = []
    child_node_2_rate_coefficients = []

    child_tr_rxn_coeffs_1 = []
    child_tr_rxn_coeffs_2 = []
    
    #let's make the parent BM node
    parent_node, _ = family.retrieve_original_entry(parent_label)
    BM_parent_rxn = Reaction(kinetics = parent_node.data)

    
    #get the dHrxns 
    
    dHrxn_1 = child_node_1_BM_rxn.get_enthalpy_of_reaction(298)
    dHrxn_2 = child_node_2_BM_rxn.get_enthalpy_of_reaction(298)

    
    for temp in temperatures: 
        
        #calculate training rxn rate
        tr_rate_1 = child_training_rxn_1.kinetics.get_rate_coefficient(temp) #J/mol #loaded in as Arr training rxn from templateRxnMap
        tr_rate_2 =child_training_rxn_2.kinetics.get_rate_coefficient(temp) #J/mol #loaded in as Arr training rxn from templateRxnMap

        child_tr_rxn_coeffs_1.append(tr_rate_1)
        child_tr_rxn_coeffs_2.append(tr_rate_2)
        
        #calculate BM child rates
        BM_child_rate_1 = child_node_1_BM_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_1) #loaded in as BM kinetics 
        BM_child_rate_2 = child_node_2_BM_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_2) #loaded in as BM kinetics 

        child_node_1_rate_coefficients.append(BM_child_rate_1)
        child_node_2_rate_coefficients.append(BM_child_rate_2)
        
        #calculate BM for parent node at both dHrxns for child rxns
        BM_parent_node_rate_dH1 = BM_parent_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_1)
        parent_node_rate_coefficients_1.append(BM_parent_node_rate_dH1)
        BM_parent_node_rate_dH2 = BM_parent_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_2)
        parent_node_rate_coefficients_2.append(BM_parent_node_rate_dH2)
        
    
    #get lnks

    parent_node_1 = [np.log(rate) for rate in parent_node_rate_coefficients_1]
    parent_node_2 = [np.log(rate) for rate in parent_node_rate_coefficients_2]
    child_node_1 = [np.log(rate) for rate in child_node_1_rate_coefficients]  
    child_node_2 = [np.log(rate) for rate in child_node_2_rate_coefficients]
    child_tr_1 = [np.log(rate) for rate in child_tr_rxn_coeffs_1]
    child_tr_2 = [np.log(rate) for rate in child_tr_rxn_coeffs_2]


    print(child_node_1_BM_rxn.kinetics.A, child_training_rxn_1.kinetics.A)
    #plotting
    plt.figure()   
    plt.plot(inverse_temps, child_node_1, '-', c= 'r', label = 'child BM rxn 1')
    plt.plot(inverse_temps, child_node_2, '-', c= 'g', label = 'child BM rxn 2')
    plt.plot(inverse_temps, child_tr_1, 'o', c='r', label='child training rxn 1')
    plt.plot(inverse_temps, child_tr_2, 'o', c='g', label='child training rxn 2')
    plt.plot(inverse_temps, parent_node_1, ':', c='r', label = 'parent BM fit 1')   
    plt.plot(inverse_temps, parent_node_2, ':', c='g', label = 'parent BM fit 2')   


    plt.title(parent_label)
    plt.xlabel('1000/T')
    plt.ylabel('ln(k)')
    plt.legend()
    plt.show()

In [ ]:
#n = 4, node with 4 reactions
%matplotlib inline
temperatures = [ 300., 500.,  600.,  700. , 800.,  900., 1000., 1100., 1200., 1500., 2000.]
inverse_temps = [1000/T for T in temperatures]



for parent_label, children_labels in parent_to_child_4.items():
    
    training_rxns = templateRxnMap[parent_label] 
    for tr_rxn in training_rxns:
        print(tr_rxn)
    BM_entries = [family.retrieve_original_entry(label)[0].data for label in children_labels]

    
    matches = []
    matched_BM = []
    matched_tr = []

    for BM_kinetics in BM_entries:
        for tr_rxn in training_rxns:
            BM_str_of_float = re.search('([1-9]\.?[0-9][0-9])', str(BM_kinetics.A.value_si)).group(1)
            tr_str_of_float = re.search('([1-9]\.?[0-9][0-9])', str(tr_rxn.kinetics.A.value_si)).group(1)
            if BM_str_of_float==tr_str_of_float:
                print('yeah')
                matches.append((Reaction(reactants=tr_rxn.reactants, products=tr_rxn.products, kinetics = BM_kinetics), tr_rxn))
                matched_BM.append(BM_kinetics)
                matched_tr.append(tr_rxn)
                
    if len(matches)==0:
        print("couldn't find any matches")
        continue
        #skipping because the A's are probably very different
        
    for (BM, tr) in matches:
        print(BM.kinetics.A.value_si, tr.kinetics.A.value_si)
        
#     if 0<len(matches)<len(training_rxns) and len(BM_entries)==len(training_rxns):
#         leftover_tr_rxns = [tr_rxn for tr_rxn in training_rxns if tr_rxn not in matched_tr]
#         leftover_BM_kinetics = [BM_kinetics for BM_kinetics in BM_entries if BM_kinetics not in matched_BM]
#         for leftover_tr_rxn in leftover_tr_rxns:
#             for leftover_BM_kinetics in leftover_BM_kinetics_:
#                 if re.search('([1-9]\.?[0-9][0-9])', str(leftover_BM_kinetics)).group(1)==re.search('([1-9]\.?[0-9][0-9])', str(leftover_tr_rxn.kinetics.A.value_si)).group(1):
#                     leftover_BM_reaction = Reaction(reactants=leftover_tr_rxn.reactants, products=leftover_tr_rxn.products, kinetics = leftover_BM_kinetics)
#                     print(leftover_BM_reaction.kinetics.A, leftover_tr_rxn.kinetics.A)
#                     matches.append((leftover_BM_reaction, leftover_tr_rxn))

    if len(BM_entries)!=len(matches):
        flag=False
    else:
        flag=True

    child_node_1_BM_rxn, child_training_rxn_1= matches[0] 
    child_node_2_BM_rxn, child_training_rxn_2 = matches[1] 
    child_node_3_BM_rxn, child_training_rxn_3= matches[2]
    if flag==True: 
        child_node_4_BM_rxn, child_training_rxn_4 = matches[3] 
    
    #empty lists for collecting data
    parent_node_rate_coefficients_1 = []
    parent_node_rate_coefficients_2 = []
    parent_node_rate_coefficients_3 = []
    parent_node_rate_coefficients_4 = []

    child_node_1_rate_coefficients = []
    child_node_2_rate_coefficients = []
    child_node_3_rate_coefficients = []
    child_node_4_rate_coefficients = []

    child_tr_rxn_coeffs_1 = []
    child_tr_rxn_coeffs_2 = []
    child_tr_rxn_coeffs_3 = []
    child_tr_rxn_coeffs_4 = []
    
    #let's make the parent BM node
    parent_node, _ = family.retrieve_original_entry(parent_label)
    BM_parent_rxn = Reaction(kinetics = parent_node.data)

    
    #get the dHrxns 
    
    dHrxn_1 = child_node_1_BM_rxn.get_enthalpy_of_reaction(298)
    dHrxn_2 = child_node_2_BM_rxn.get_enthalpy_of_reaction(298)
    dHrxn_3 = child_node_3_BM_rxn.get_enthalpy_of_reaction(298)
    if flag==True: 
        dHrxn_4 = child_node_4_BM_rxn.get_enthalpy_of_reaction(298)

    
    for temp in temperatures: 
        
        #calculate training rxn rate
        tr_rate_1 = child_training_rxn_1.kinetics.get_rate_coefficient(temp) #J/mol #loaded in as Arr training rxn from templateRxnMap
        tr_rate_2 =child_training_rxn_2.kinetics.get_rate_coefficient(temp) #J/mol #loaded in as Arr training rxn from templateRxnMap
        tr_rate_3 = child_training_rxn_3.kinetics.get_rate_coefficient(temp) #J/mol #loaded in as Arr training rxn from templateRxnMap
        if flag==True: 
            tr_rate_4 =child_training_rxn_4.kinetics.get_rate_coefficient(temp) #J/mol #loaded in as Arr training rxn from templateRxnMap

        child_tr_rxn_coeffs_1.append(tr_rate_1)
        child_tr_rxn_coeffs_2.append(tr_rate_2)
        child_tr_rxn_coeffs_3.append(tr_rate_3)
        if flag==True:
            child_tr_rxn_coeffs_4.append(tr_rate_4)
        
        #calculate BM child rates
        BM_child_rate_1 = child_node_1_BM_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_1) #loaded in as BM kinetics 
        BM_child_rate_2 = child_node_2_BM_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_2) #loaded in as BM kinetics 
        BM_child_rate_3 = child_node_3_BM_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_3) #loaded in as BM kinetics 
        if flag==True:
            BM_child_rate_4 = child_node_4_BM_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_4) #loaded in as BM kinetics 

        child_node_1_rate_coefficients.append(BM_child_rate_1)
        child_node_2_rate_coefficients.append(BM_child_rate_2)
        child_node_3_rate_coefficients.append(BM_child_rate_3)
        if flag==True:
            child_node_4_rate_coefficients.append(BM_child_rate_4)
        
        #calculate BM for parent node at both dHrxns for child rxns
        BM_parent_node_rate_dH1 = BM_parent_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_1)
        parent_node_rate_coefficients_1.append(BM_parent_node_rate_dH1)
        BM_parent_node_rate_dH2 = BM_parent_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_2)
        parent_node_rate_coefficients_2.append(BM_parent_node_rate_dH2)
        BM_parent_node_rate_dH3 = BM_parent_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_3)
        parent_node_rate_coefficients_3.append(BM_parent_node_rate_dH3)
        if flag==True:
            BM_parent_node_rate_dH4 = BM_parent_rxn.kinetics.get_rate_coefficient(temp, dHrxn=dHrxn_4)
            parent_node_rate_coefficients_4.append(BM_parent_node_rate_dH4)
        
    
    #get lnks

    parent_node_1 = [np.log(rate) for rate in parent_node_rate_coefficients_1]
    parent_node_2 = [np.log(rate) for rate in parent_node_rate_coefficients_2]
    parent_node_3 = [np.log(rate) for rate in parent_node_rate_coefficients_3]
    if flag==True:
        parent_node_4 = [np.log(rate) for rate in parent_node_rate_coefficients_4]
    
    child_node_1 = [np.log(rate) for rate in child_node_1_rate_coefficients]  
    child_node_2 = [np.log(rate) for rate in child_node_2_rate_coefficients]
    child_node_3 = [np.log(rate) for rate in child_node_3_rate_coefficients]  
    if flag==True:
        child_node_4 = [np.log(rate) for rate in child_node_4_rate_coefficients]
    
    child_tr_1 = [np.log(rate) for rate in child_tr_rxn_coeffs_1]
    child_tr_2 = [np.log(rate) for rate in child_tr_rxn_coeffs_2]
    child_tr_3 = [np.log(rate) for rate in child_tr_rxn_coeffs_3]
    if flag==True:
        child_tr_4 = [np.log(rate) for rate in child_tr_rxn_coeffs_4]


    print(child_node_1_BM_rxn.kinetics.A, child_training_rxn_1.kinetics.A)
    #plotting
    plt.figure()   
    plt.plot(inverse_temps, child_node_1, '-', c= 'r', label = 'child BM rxn 1')
    plt.plot(inverse_temps, child_node_2, '-', c= 'g', label = 'child BM rxn 2')
    plt.plot(inverse_temps, child_node_3, '-', c= 'violet', label = 'child BM rxn 3')
    if flag==True:
        plt.plot(inverse_temps, child_node_4, '-', c= 'b', label = 'child BM rxn 4')
    
    plt.plot(inverse_temps, child_tr_1, 'o', c='r', label='child training rxn 1')
    plt.plot(inverse_temps, child_tr_2, 'o', c='g', label='child training rxn 2')
    plt.plot(inverse_temps, child_tr_3, 'o', c='violet', label='child training rxn 3')
    if flag==True:
        plt.plot(inverse_temps, child_tr_4, 'o', c='b', label='child training rxn 4')
    
    plt.plot(inverse_temps, parent_node_1, ':', c='r', label = 'parent BM fit 1')   
    plt.plot(inverse_temps, parent_node_2, ':', c='g', label = 'parent BM fit 2')
    plt.plot(inverse_temps, parent_node_3, ':', c='violet', label = 'parent BM fit 3')   
    if flag==True:
        plt.plot(inverse_temps, parent_node_4, ':', c='b', label = 'parent BM fit 4')


    plt.title(parent_label)
    plt.xlabel('1000/T')
    plt.ylabel('ln(k)')
    plt.legend(loc=(1.1,0.8))
    plt.show()